**1.Install the Transformers, Datasets, and Evaluate libraries  our task**

In [1]:
# install datasets
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

**2.Download our dataset**

https://huggingface.co/datasets/AmazonScience/massive/viewer/en-US/train

In [3]:
#load the dataset
from datasets import load_dataset
source = "AmazonScience/massive"
#AmazonScience massive en-US dataset
dataset = load_dataset(source, "en-US", download_mode="force_redownload")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments'],
        num_rows: 2974
    })
})


 **3.Explore dataset**

In [4]:
dataset["train"].features

{'id': Value(dtype='string', id=None),
 'locale': Value(dtype='string', id=None),
 'partition': Value(dtype='string', id=None),
 'scenario': ClassLabel(names=['social', 'transport', 'calendar', 'play', 'news', 'datetime', 'recommendation', 'email', 'iot', 'general', 'audio', 'lists', 'qa', 'cooking', 'takeaway', 'music', 'alarm', 'weather'], id=None),
 'intent': ClassLabel(names=['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock', 'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe', 'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up', 'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup', 'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query', 'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query', 'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim', 'calendar_query', 'em

In [5]:
dataset["train"][:2]

{'id': ['1', '2'],
 'locale': ['en-US', 'en-US'],
 'partition': ['train', 'train'],
 'scenario': [16, 16],
 'intent': [48, 48],
 'utt': ['wake me up at nine am on friday',
  'set an alarm for two hours from now'],
 'annot_utt': ['wake me up at [time : nine am] on [date : friday]',
  'set an alarm for [time : two hours from now]'],
 'worker_id': ['1', '1'],
 'slot_method': [{'slot': [], 'method': []}, {'slot': [], 'method': []}],
 'judgments': [{'worker_id': [],
   'intent_score': [],
   'slots_score': [],
   'grammar_score': [],
   'spelling_score': [],
   'language_identification': []},
  {'worker_id': [],
   'intent_score': [],
   'slots_score': [],
   'grammar_score': [],
   'spelling_score': [],
   'language_identification': []}]}

In [6]:
dataset["validation"][:2]

{'id': ['11', '17'],
 'locale': ['en-US', 'en-US'],
 'partition': ['dev', 'dev'],
 'scenario': [8, 8],
 'intent': [40, 31],
 'utt': ['turn the lights off please', 'dim the lights in the hall'],
 'annot_utt': ['turn the lights off please',
  'dim the lights in the [house_place : hall]'],
 'worker_id': ['0', '1'],
 'slot_method': [{'slot': [], 'method': []}, {'slot': [], 'method': []}],
 'judgments': [{'worker_id': [],
   'intent_score': [],
   'slots_score': [],
   'grammar_score': [],
   'spelling_score': [],
   'language_identification': []},
  {'worker_id': [],
   'intent_score': [],
   'slots_score': [],
   'grammar_score': [],
   'spelling_score': [],
   'language_identification': []}]}

In [7]:
dataset["test"][:2]

{'id': ['0', '3'],
 'locale': ['en-US', 'en-US'],
 'partition': ['test', 'test'],
 'scenario': [16, 10],
 'intent': [48, 46],
 'utt': ['wake me up at five am this week', 'quiet'],
 'annot_utt': ['wake me up at [time : five am] [date : this week]', 'quiet'],
 'worker_id': ['1', '1'],
 'slot_method': [{'slot': [], 'method': []}, {'slot': [], 'method': []}],
 'judgments': [{'worker_id': [],
   'intent_score': [],
   'slots_score': [],
   'grammar_score': [],
   'spelling_score': [],
   'language_identification': []},
  {'worker_id': [],
   'intent_score': [],
   'slots_score': [],
   'grammar_score': [],
   'spelling_score': [],
   'language_identification': []}]}

**4.Load the tokenizer with the appropiate checkpoint and tokenization of dataset**

In [8]:
from transformers import AutoTokenizer # AutoTokenizer from fransformers library

checkpoint = "bert-base-uncased"# Bert checkpoint that we ll use for the tokenizer and model later
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#function the call the pretrained tokenizer our data ( column utt)
def tokenize_function(example):
    return tokenizer(example["utt"],  truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)# tokenization

Map:   0%|          | 0/11514 [00:00<?, ? examples/s]

Map:   0%|          | 0/2033 [00:00<?, ? examples/s]

Map:   0%|          | 0/2974 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets["train"].column_names

['id',
 'locale',
 'partition',
 'scenario',
 'intent',
 'utt',
 'annot_utt',
 'worker_id',
 'slot_method',
 'judgments',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [10]:
#change column name from "intent" to "labels"
tokenized_datasets = tokenized_datasets.rename_column("intent", "labels")


In [11]:
#load datacollar function in order to perform dynamic padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
"""
use the function to_tf_dataset from tensorflow to perfom batching and collate
the dataset and  immediately converts it to a format that’s ready for training
"""
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=16,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [13]:
num_labels=len(np.unique(np.array(tokenized_datasets["train"]['labels'])))

**5.Load transformers model with the checkpoint define above and with the number of classes**

In [14]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**6.Define the learning rate scheduler to have it decreasing after each iterations**

In [15]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 16
num_epochs = 5
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=3e-5,
    end_learning_rate=0.0,
    decay_steps=num_train_steps,
    power=1
)
num_train_steps

3600

**7.Load Adam optimizer**

the parameters used were taken from online litterature

In [16]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler, beta_1=.9, beta_2=.999,epsilon=1e-08)


**8.compile the model**

In [17]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(
    optimizer= opt,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


**9.Fit the model to our dataset with a validation set**

In [18]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,epochs=num_epochs,verbose=True
)

Epoch 1/5
720/720 [==============================] - 157s 143ms/step - loss: 1.8464 - accuracy: 0.6222 - val_loss: 0.7899 - val_accuracy: 0.8323
Epoch 2/5
720/720 [==============================] - 100s 138ms/step - loss: 0.5832 - accuracy: 0.8829 - val_loss: 0.5531 - val_accuracy: 0.8657
Epoch 3/5
720/720 [==============================] - 82s 113ms/step - loss: 0.2989 - accuracy: 0.9450 - val_loss: 0.5205 - val_accuracy: 0.8770
Epoch 4/5
720/720 [==============================] - 83s 116ms/step - loss: 0.1874 - accuracy: 0.9670 - val_loss: 0.4876 - val_accuracy: 0.8864
Epoch 5/5
720/720 [==============================] - 81s 113ms/step - loss: 0.1325 - accuracy: 0.9791 - val_loss: 0.4812 - val_accuracy: 0.8879


**10.Predict and check the perfomance of our fine tune model**


In [19]:
preds = model.predict(tf_validation_dataset)["logits"]

128/128 [==============================] - 9s 43ms/step


In [20]:
class_preds = np.argmax(preds, axis=1)

In [22]:
from sklearn.metrics import accuracy_score  ,f1_score
accuracy = accuracy_score(dataset["validation"]["intent"], class_preds)
F1_score = f1_score(dataset["validation"]["intent"], class_preds,average="weighted")
print('Accuracy: {}'.format(round(accuracy,3)),'F1 score: {}'.format(round(F1_score,3)))

Accuracy: 0.888 F1 score: 0.886


**predict classes**

In [29]:
sample= ["what's happening in football today",
         "reply to patrick that i am on my way",
         "what an umbrella means",
         "i m sick "]

In [30]:
sample_tokenized=tokenizer(sample,padding=True,truncation=True,return_tensors="tf")
sample_tokenized_tuple = (sample_tokenized['input_ids'], sample_tokenized['token_type_ids'], sample_tokenized['attention_mask'])
sample_preds=model.predict(sample_tokenized_tuple)["logits"]
#sample_preds
sample_preds = np.argmax(sample_preds,axis=1)
sample_preds
print(*(dataset["train"].features['intent'].int2str(sample_preds)),sep = "\n")

1/1 [==============================] - 0s 49ms/step
news_query
email_sendemail
qa_definition
general_quirky
